In [26]:
from flask import Flask, request, jsonify
from typing import Any, Text, Dict, List, Tuple
import requests

# --- Dependencias RAG (De LangChain) ---
# Usamos langchain_community porque Flask solo necesita interactuar con los objetos
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.schema import Document

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# --- CONFIGURACIÓN RAG LOCAL ---
# Modelo LLM que debe estar corriendo en Ollama (ej: ollama pull phi3)
OLLAMA_MODEL_NAME = "phi3"
OLLAMA_BASE_URL = "http://localhost:11434"
RAG_API_PORT = 8000


In [29]:
def initialize_rag_chain():
    """
    Construye la cadena RAG usando LLM y Embeddings del mismo servidor Ollama.
    """
    print("Inicializando LLM Local (Ollama) y Base Vectorial...")
    
    # 1. Conectores LLM y Embeddings (Ambos usan Ollama y phi3)
    try:
        requests.get(OLLAMA_BASE_URL)
        
        # GENERACIÓN: Usa el modelo Phi-3 para crear la respuesta
        llm = OllamaLLM(model=OLLAMA_MODEL_NAME, base_url=OLLAMA_BASE_URL, temperature=0.1)
        
        # EMBEDDINGS: Usa el mismo modelo Phi-3 (o un modelo compatible) para vectorizar los documentos.
        embeddings = OllamaEmbeddings(model=OLLAMA_MODEL_NAME, base_url=OLLAMA_BASE_URL)
        
    except Exception as e:
        print(f"Error al conectar a Ollama: {e}")
        return None

    # 2. Base de Conocimiento de Ejemplo
    docs = [
        {"page_content": "Para reiniciar tu router, desconecta el cable de alimentación por 30 segundos y vuelve a conectarlo. Espera 2 minutos para que se estabilice la conexión."},
        {"page_content": "Si el internet está lento, verifica las luces. Si la luz 'Internet' está roja, hay una falla en la línea."},
    ]
    documents = [Document(page_content=t["page_content"]) for t in docs]
    vectorstore = Chroma.from_documents(documents, embeddings)
    retriever = vectorstore.as_retriever()
    
    # 3. PROMTPTS y CADENAS MODULARES 
    
    # Prompt 1: Genera la consulta de búsqueda
    query_generator_prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Basado en la conversación, genera una consulta de búsqueda en español que resuelva la duda."),
    ])
    
    # Cadena 1: Recuperador consciente del historial
    retriever_chain = create_history_aware_retriever(llm, retriever, query_generator_prompt)

    # Prompt 2: Genera la respuesta final con el contexto
    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", "Eres un agente de soporte técnico de El Salvador. Responde la pregunta del usuario basándote solo en el siguiente contexto:\n\n{context}\nSi el contexto no proporciona la respuesta, indica que no la sabes. Utiliza un tono amigable y salvadoreño."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ])
    
    # Cadena 2: Combina los documentos y el LLM para generar la respuesta
    document_chain = create_stuff_documents_chain(llm, answer_prompt)

    # 4. Cadena Final: Combina la recuperación y la generación (Resuelve el AttributeError)
    qa_chain = create_retrieval_chain(retriever_chain, document_chain)
    
    print(f"Inicialización RAG Local completa. LLM: {OLLAMA_MODEL_NAME}")
    return qa_chain

In [30]:

# Inicializar la cadena RAG globalmente
try:
    RAG_CHAIN = initialize_rag_chain()
except ConnectionError as e:
    # Si la conexión falla, se detiene la carga del servidor
    print(e)
    RAG_CHAIN = None

Inicializando LLM Local (Ollama) y Base Vectorial...
Inicialización RAG Local completa. LLM: phi3


In [34]:
question = "¿Cómo puedo reiniciar mi router?"
chat_history = [("user", ": Mi internet está muy lento últimamente. ¿Qué puedo hacer?")]

result = RAG_CHAIN.invoke({"input": question, "chat_history": chat_history})
response_text = result["answer"]
print("Respuesta RAG:", response_text)

Respuesta RAG: Primero, por favor verifica las luces de tu conexión a Internet para saber si la 'luz Internet' está roja o verde. Si es roja, esto indica que hay un problema en la línea y necesitarás llamar al número de atención técnica 2255-6677 para reportarlo.

Si la luz aparece verde, el problema podría estar relacionado con tu dispositivo o su configuración interna. En este caso, también te recomendamos que llames al mismo número de atención técnica 2255-6677 para recibir asistencia especializada y solucionar cualquier inconveniente desde ahí.

Si necesitas reiniciar tu router, por favor sigue estos pasos:
1. Desconéctalo del enchufe de corriente principal durante unos 30 segundos a un minuto para permitir que se descargue completamente la energía residual.
2. Conecta el router nuevamente al enchufe y espera algunos momentos mientras reconecta con tu red Wi-Fi (puede tardar de 1 a 5 minutos).
3. Una vez reconectado, verifica si tu velocidad de internet ha mejorado o no. Si persist